You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!uv tool install crewai crewai_tools langchain_community
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew
from crewai import LLM
from dotenv import load_dotenv
import os

load_dotenv()  # this will load env keys from .env file

True

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see its documentation at https://docs.crewai.com/concepts/llms

In [3]:
api_key = os.getenv('GROQ_API_KEY')
llm = LLM(
    model="groq/gemma2-9b-it",
    temperature = 0.7
)

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm=llm
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the 'Content Writer'. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the written blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=False
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
##  Content Plan: Demystifying Artificial Intelligence 

**Goal:** To provide a clear, engaging, and informative overview of Artificial Intelligence, its latest trends, and its potential impact on our lives.

**Target Audience:**

* **Tech enthusiasts:**  Individuals interested in the latest technological advancements and their implications.
* **Professionals:** Individuals across various industries seeking to understand how AI is transforming their field.
* **General public:** Individuals curious about AI and its potential benefits and risks.

**Pain Points:**

* **Lack of understanding:*

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result.raw)

## Demystifying Artificial Intelligence: Trends, Impact, and the Future

Artificial intelligence (AI) has rapidly transitioned from a futuristic concept to a tangible force shaping our world. From personalized recommendations on streaming platforms to self-driving cars, AI is quietly revolutionizing industries and touching nearly every aspect of our lives. But what exactly is AI, and how is it impacting us today? 

**Beyond the Buzzword: Defining AI**

At its core, AI refers to the ability of computers to perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. Unlike traditional programming, which relies on explicit instructions, AI algorithms learn from data, identifying patterns and making predictions. This ability to learn and adapt makes AI a powerful tool with the potential to automate complex processes, uncover hidden insights, and create entirely new possibilities.

**AI is Everywhere: A Look at Today's Trends**

The field of AI is constantly evolving, with new breakthroughs and applications emerging at a rapid pace. Some of the most notable trends shaping the AI landscape include:

* **Generative AI:** This exciting branch of AI focuses on creating new content, such as text, images, music, and even code.  Models like ChatGPT and DALL-E have captured public imagination by generating human-quality text and images, respectively. These tools have vast potential in creative fields, content creation, and even scientific research.
* **Explainable AI (XAI):** As AI systems become more complex, understanding how they arrive at their decisions is crucial. XAI aims to make AI more transparent and interpretable, allowing humans to better understand the reasoning behind AI-driven outcomes. This is particularly important in fields like healthcare and finance where trust and accountability are paramount.
* **Edge AI:** Traditionally, AI relied on powerful cloud computing infrastructure for processing. Edge AI, however, brings AI processing to the edge of the network, closer to the data source. This reduces latency, enhances privacy, and enables real-time decision-making in applications like autonomous vehicles and industrial automation.

**Shaping the Future: Key Players in AI**

The development and advancement of AI are driven by a diverse ecosystem of players:

* **Tech Giants:** Companies like Google, Microsoft, Amazon, and Meta are investing heavily in AI research and development, pushing the boundaries of what's possible with AI. Their vast resources and data allow them to train powerful AI models and develop innovative applications across various industries.
* **Startups:**  A wave of startups is emerging, focusing on niche applications of AI in fields like healthcare, finance, education, and agriculture. Their agility and focus on specific problems often lead to innovative solutions that address real-world challenges.
* **Research Institutions:** Universities and research labs play a vital role in advancing fundamental AI research. Their work on areas like machine learning, deep learning, and robotics lays the groundwork for future AI breakthroughs and applications.

**Navigating the Horizon: The Future of AI**

The future of AI holds both immense promise and significant challenges. While AI has the potential to revolutionize healthcare, education, sustainability, and countless other aspects of our lives, it also raises concerns about job displacement, algorithmic bias, and the potential misuse of this powerful technology.

It is crucial that we approach AI development and deployment responsibly, ensuring that it benefits society as a whole. This requires ongoing dialogue, collaboration, and a commitment to ethical considerations. As AI continues to evolve, our role is to shape its development in a way that aligns with our values and creates a better future for all.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "Agenic AI Trend."
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result.raw)